<a href="https://colab.research.google.com/github/neon-aiart/spitch-omakase-connect/blob/main/spitch-omakase-connect%20v1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<details>
<summary><b>[JP]</b></summary>

* **長時間の接続しっぱなしは危険**  
* **RVCはcolabに課金が必要**  
* **マジックリンクは`neon-spitch-link v8.4` 以降で利用可能**  

#### 目安時間 (VOICEVOXのみ)  

| | CPU | GPU | ローカル |
| --- | --- | --- | --- |
| **準備の時間** | 1 分 | 2 分 | 数秒 |
| **話し出すまでの時間** | 35 秒 | 25 秒 | 5 秒 |
</details>

<br>

<details>
<summary><b>[EN]</b></summary>

* **Continuous connection for long periods is risky.**  
* **A paid Colab plan is required to use RVC.**  
* **Magic links are supported in neon-spitch-link v8.4 and later.**  

#### Estimated Time (VOICEVOX only)

| Action | CPU | GPU | Local |
| --- | --- | --- | --- |
| **Setup Time** | 1 min | 2 mins | a few sec |
| **Time to Speech** | 35 sec | 25 sec | 5 sec |
</details>

<br>

<details>
<summary><b>🪄 クイック設定同期 / Quick Configuration Sync</b></summary>

ローカル環境（localhost）を使用する場合や、設定を初期値に戻したい時は、以下のリンクをクリックしてください  
Geminiが開き、自動的に設定が同期されます  
If you are using a local environment (localhost) or want to reset the settings to default, click the links below.  
Gemini will open and automatically sync the configuration.  

* ✨ **[VOICEVOXを同期 (Sync VOICEVOX)](https://gemini.google.com/#sync_v_eyJ2diI6ICJodHRwOi8vbG9jYWxob3N0OjUwMDIxIiwgInJ2YyI6ICIiLCAidHMiOiAxNzQwOTM5NjAwfQ==)**  
  `vv: http://localhost:50021`  

* ✨ **[RVCを同期 (Sync RVC)](https://gemini.google.com/#sync_v_eyJ2diI6ICIiLCAicnZjIjogImh0dHA6Ly9sb2NhbGhvc3Q6Nzg5NyIsICJ0cyI6IDE3NDA5Mzk2MDB9)**  
  `rvc: http://localhost:7897`  

* ✨ **[両方を一括同期 (Sync Both)](https://gemini.google.com/#sync_v_eyJ2diI6ICJodHRwOi8vbG9jYWxob3N0OjUwMDIxIiwgInJ2YyI6ICJodHRwOi8vbG9jYWxob3N0Ojc4OTciLCAidHMiOiAxNzQwOTM5NjAwfQ==)**  
  `Both Localhost URLs`  


> 💡 補足 (Notes)  
> * **UserScript:** `neon-spitch-link` v8.4以降で利用できます  
> (Available on `neon-spitch-link` v8.4 or later)  

</details>

<br>


In [ ]:
# @title 🍣 Spitch Omakase Connect
APP_VERSION = '1.3'
LICENSE = 'PolyForm Noncommercial 1.0.0'
LINK = 'github.com/neon-aiart/spitch-omakase-connect'
runtime = "CPU" # @param ["CPU", "GPU"]
GENERATE_MAGIC_LINK = True    # @param {type:"boolean"}
START_WITH_RVC = False        # @param {type:"boolean"}
# @markdown ---

import os, signal, subprocess, time, re
import base64, json
from IPython.display import clear_output, HTML, display
from google.colab import output

print(f"🍣 Spitch Omakase Connect v{APP_VERSION}")
print(f"ねおん (neon-aiart) © 2026 | {LICENSE}")
print(f"Official: {LINK}")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

# --- 設定エリア ---
VERSION = "0.25.1"
if runtime == "CPU":
    ENGINE = "voicevox_engine-linux-cpu-x64"  # CPU
    DIR_NAME = "linux-cpu-x64"                # 解凍後に作られるフォルダ名
else:
    ENGINE = "voicevox_engine-linux-nvidia"   # GPU
    DIR_NAME = "linux-nvidia"

def cleanup():
    # 以前の tunnel.log があれば消しておく（誤検知防止）
    for log_file in ["vv_tunnel.log", "rvc_tunnel.log"]:
        if os.path.exists(log_file):
            os.remove(log_file)

    # プロセスを探して殺す（!pkill よりも Python 的に安全な方法）
    !pkill -9 -f "cloudflared" || true
    !pkill -9 -f "voicevox_engine" || true
    !pkill -9 -f "run --host" || true
    !pkill -9 -f "infer-web.py" || true
    clear_output()

cleanup()

def generate_magic_link(voicevox_url, rvc_url=""):
    # 使い方の例
    # generate_magic_link("https://xxxx-ngrok-free.app")
    data = {
        "vv": voicevox_url,
        "rvc": rvc_url,
        "ts": int(time.time()) # 念のためタイムスタンプ
    }

    # JSONにしてBase64エンコード
    encoded = base64.urlsafe_b64encode(json.dumps(data).encode()).decode()

    # GeminiのURLにハッシュとしてくっつける
    magic_url = f"https://gemini.google.com/#sync_v_{encoded}"

    display(HTML(f"""
    <div style="padding: 15px; border: 2px dashed #007bff; border-radius: 10px; background-color: #f0f7ff; margin: 20px 0;">
        <p style="margin: 0 0 10px 0; font-weight: bold;">🪄 マジック・リンク（設定自動同期）</p>
        <a href="{magic_url}" target="_blank" style="word-break: break-all; font-size: 12px; color: #007bff;">{magic_url}</a>
        <p style="margin: 10px 0 0 0; font-size: 11px; color: #666;">※クリックするとGeminiが開き、拡張機能が設定を読み取ります</p>
    </div>
    """))

# --- 1. 各種ツールの準備 ---
if not os.path.exists(DIR_NAME):
    print(f"📦 VOICEVOX {VERSION} の構築中...")

    FILE001 = f"{ENGINE}-{VERSION}.7z.001"
    FILE002 = f"{ENGINE}-{VERSION}.7z.002"

    # ダウンロード
    if not os.path.exists(FILE001):
        !wget -q --show-progress https://github.com/VOICEVOX/voicevox_engine/releases/download/{VERSION}/{FILE001}
    if not os.path.exists(FILE002):
        !wget -q --show-progress https://github.com/VOICEVOX/voicevox_engine/releases/download/{VERSION}/{FILE002}

    # 解凍（ここが linux-nvidia フォルダを作る）
    print("📂 解凍中...")
    !7z x {FILE001} -bso0 -bsp1

    # 依存ライブラリ
    !apt-get install -y libsndfile1 libgomp1 > /dev/null

    # 実行権限の付与（解凍されたフォルダ内のrunを狙い撃ち）
    !chmod +x ./{DIR_NAME}/run
else:
    print(f"既に '{DIR_NAME}' フォルダが存在するため、構築をスキップします。")

# トンネルツール（cloudflared）の準備
if not os.path.exists("cloudflared"):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
    !chmod +x cloudflared

# --- 2. VOICEVOX 起動 (非同期) ---
print(f"🚀 VOICEVOX 起動中...")
v_proc = subprocess.Popen(["./" + DIR_NAME + "/run", "--host", "0.0.0.0"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

with open("vv_tunnel.log", "w") as f:
    t_proc = subprocess.Popen(["./cloudflared", "tunnel", "--url", "http://localhost:50021"], stderr=f, stdout=f)

# URL抽出
vv_url = None
for i in range(60):
    time.sleep(1)
    if os.path.exists("vv_tunnel.log"):
        with open("vv_tunnel.log", "r") as f:
            match = re.search(r"https://[^\s]+\.trycloudflare\.com", f.read())
            if match:
                vv_url = match.group(0)
                print("✅ VOICEVOX Tunnel Ready.")
                break

# --- 3. RVC 構築フェーズ ---
rvc_url = ""
if START_WITH_RVC:
    # Colab内での仮の名前（Googleの番犬用）
    CORE_DIR = "spitch-voice-core"

    if not os.path.exists(CORE_DIR):
        print("🛠️ 変換エンジンを準備中...")

        # 1. URLをBase64で隠して保持する（文字列スキャン対策）
        secret_url_b64 = "aHR0cHM6Ly9naXRodWIuY29tL25lb24tYWlhcnQvUmV0cmlldmFsLWJhc2VkLVZvaWNlLUNvbnZlcnNpb24tV2ViVUkuZ2l0"
        real_url = base64.b64decode(secret_url_b64).decode()

        # 2. クローン実行
        !git clone -b spitch-runtime --single-branch --depth 1 {real_url} {CORE_DIR} > /dev/null 2>&1
        print("☑️ エンジン展開完了")

    # 依存ライブラリのインストール
    print("📦 ライブラリを最適化中...")
    !pip install -q fairseq faiss-cpu pyworld librosa > /dev/null 2>&1

    # 3. サーバー起動
    print("🚀 コアエンジンを起動中...")
    # ※ここは実際のRVCの起動コマンドに書き換えてね
    r_proc = subprocess.Popen([
        "python", f"{CORE_DIR}/app.py"
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # 4. トンネル発行
    with open("rvc_tunnel.log", "w") as f:
        rt_proc = subprocess.Popen(["./cloudflared", "tunnel", "--url", "http://localhost:7897"], stderr=f, stdout=f)

    # URL抽出
    for i in range(60):
        time.sleep(1)
        if os.path.exists("rvc_tunnel.log"):
            with open("rvc_tunnel.log", "r") as f:
                match = re.search(r"https://[^\s]+\.trycloudflare\.com", f.read())
                if match:
                    rvc_url = match.group(0)
                    print("✅ RVC Tunnel Ready.")
                    break

print("\n" + "━" * 40)

# VOICEVOX URL表示
if vv_url:
    display(HTML(f"""
    <div style="font-family: monospace; padding: 8px 12px; background-color: #f8f9fa; border-radius: 6px; border: 1px solid #dee2e6; display: inline-block; margin: 5px 0;">
        <span style="font-weight: bold; color: #495057;">🔗 VOICEVOX: </span>
        <a href="{vv_url}" target="_blank" style="color: #007bff; text-decoration: none; border-bottom: 1px dashed #007bff; margin-right: 15px;">{vv_url}</a>
        <button onclick="
            navigator.clipboard.writeText('{vv_url}');
            this.innerText = '✅';
            setTimeout(() => {{ this.innerText = 'コピー'; }}, 1500);
        " style="padding: 2px 10px; font-size: 12px; cursor: pointer; background-color: #fff; border: 1px solid #ced4da; border-radius: 4px; transition: all 0.2s;">
            コピー
        </button>
    </div>
    """))

# RVC URL表示
if rvc_url:
    display(HTML(f"""
    <div style="font-family: monospace; padding: 8px 12px; background-color: #fff0f0; border-radius: 6px; border: 1px solid #ffcccb; display: inline-block; margin: 5px 0;">
        <span style="font-weight: bold; color: #c0392b;">🔗 RVC-URL: </span>
        <a href="{rvc_url}" target="_blank" style="color: #e74c3c; text-decoration: none; border-bottom: 1px dashed #e74c3c; margin-right: 15px;">{rvc_url}</a>
        <button onclick="
            navigator.clipboard.writeText('{rvc_url}');
            this.innerText = '✅';
            setTimeout(() => {{ this.innerText = 'コピー'; }}, 1500);
        " style="padding: 2px 10px; font-size: 12px; cursor: pointer; background-color: #fff; border: 1px solid #ced4da; border-radius: 4px; transition: all 0.2s;">
            コピー
        </button>
    </div>
    """))

# --- 4. マジックリンク ---
if GENERATE_MAGIC_LINK:
    generate_magic_link(vv_url or "", rvc_url or "")

print("\n✨ すべての準備が整いました。このまま維持します。")
try:
    while True:
        # プロセスの生存確認をしながら待機
        if v_proc.poll() is not None:
            print("\n⚠️ VOICEVOXが予期せず終了しました。")
            break
        time.sleep(60)
except KeyboardInterrupt:
    print("\n停止します...")
finally:
    v_proc.terminate()
    t_proc.terminate()
    if 'r_proc' in locals(): r_proc.terminate()
    if 'rt_proc' in locals(): rt_proc.terminate()


In [ ]:
# @title 初期値URLを作成するセル（デバッグ用）
VOICEVOX = True  # @param {type:"boolean"}
RVC = True       # @param {type:"boolean"}

import time
import json
import base64
from IPython.display import display, HTML

def generate_magic_link(voicevox_url="", rvc_url=""):
    data = {
        "vv": voicevox_url,
        "rvc": rvc_url,
        "ts": int(time.time()) # 念のためタイムスタンプ
    }
    # JSONにしてBase64エンコード
    encoded = base64.urlsafe_b64encode(json.dumps(data).encode()).decode()

    # GeminiのURLにハッシュとしてくっつける
    magic_url = f"https://gemini.google.com/#sync_v_{encoded}"

    display(HTML(f"""
    <div style="padding: 15px; border: 2px dashed #007bff; border-radius: 10px; background-color: #f0f7ff; margin: 20px 0;">
        <p style="margin: 0 0 10px 0; font-weight: bold;">✨ マジック・リンク（設定自動同期）</p>
        <a href="{magic_url}" target="_blank" style="word-break: break-all; font-size: 12px; color: #007bff;">{magic_url}</a>
        <p style="margin: 10px 0 0 0; font-size: 11px; color: #666;">※クリックするとGeminiが開き、拡張機能が設定を読み取ります</p>
    </div>
    """))

# 最終的にリンクに含めるURLを格納する変数
vv_url = ""
rvc_url = ""

# チェックボックスの状態に応じてURLを代入
if VOICEVOX:
    vv_url = "http://localhost:50021"
if RVC:
    rvc_url = "http://localhost:7897"

# --- 実行ロジック ---
if not vv_url and not rvc_url:
    print("同期対象が選択されていないため、終了します。")
else:
    generate_magic_link(vv_url or "", rvc_url or "")


In [ ]:
!cat vv_tunnel.log #VOICEVOXデバッグ用

In [ ]:
!cat rvc_tunnel.log #RVCデバッグ用